# Building an SQLite database from the export

In [ ]:
from signal_backup_manager import importer
import sqlite3
import numpy as np
import ctypes

In [ ]:
BACKUP_PASSPHRASE = '770374738950407044994423070922'
BACKUP_FILE       = '../res/signal.backup'

bkp = importer.SignalBackupImporter(BACKUP_FILE, BACKUP_PASSPHRASE)

In [ ]:
bkp.db_version

In [ ]:
statement = bkp.db_statements[100]
params = ()
for param in statement.parameters:
    if param.HasField('stringParamter'):
        params += (param.stringParamter,)  
    if param.HasField('doubleParameter'):
        params += (param.doubleParameter,)
    if param.HasField('integerParameter'):
        params += (param.integerParameter,)
    if param.HasField('nullparameter'):
        params += (param.nullparameter,)

In [ ]:
params

In [ ]:
bkp.db_statements[300]

In [ ]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

for statement in bkp.db_statements:
    
    params = ()
    for param in statement.parameters:
        if param.HasField('stringParamter'):
            params += (param.stringParamter,)  
        if param.HasField('doubleParameter'):
            params += (param.doubleParameter,)
        if param.HasField('integerParameter'):
            if param.integerParameter > 2 ** 63:
                params += (ctypes.c_long(param.integerParameter).value,)
            else:
                params += (param.integerParameter,)
        if param.HasField('blobParameter'):
            params += (param.blobParameter,)
        if param.HasField('nullparameter'):
            params += (param.nullparameter,)
    try:
        c.execute(statement.statement, params)
    except:
        print(statement)
        print(params)
        conn.close()
        raise

conn.commit()
conn.close()

In [ ]:
bkp.db_preferences